# Tensorlines in 3D

Given the eigenvalue problem:

\begin{equation}
\mathbf{A} \mathbf{e}_k = \lambda_k \mathbf{e}_k, \quad k = 1, 2, 3
\label{eq: eigenvalueproblem}
\end{equation}

where we assume $ A(\mathbf{x}, t) = A(\mathbf{x}, t)^T \in \mathbb{R}^{3 \times 3}, \mathbf{e}_k(\mathbf{x}, t) \in \mathbb{R}^3 $ and $ \lambda_k(\mathbf{x}, t) \in \mathbb{R} $. We seek to find a way to compute the tensorlines of $ \mathbf{A} $ without having the deal with the orientational discontinuities of the eigenvector field $ \mathbf{e}_{k}(\mathbf{x},t) $.

For a fixed time $ t $ and a fixed integer $ k $ a tensorline is a parametrized curve $ \mathbf{x}(s) \in \mathbb{R}^3 $ such that $ \mathbf{x}^{\prime}(s)=\mathbf{e}_{k}(\mathbf{x}(s),t) $ for some k. Since $ \mathbf{e}_{k} $ is of unit length, we can view it as a vector spanning the surface of the unit sphere in $ \mathbf{R}^{3} $. Therefore, using the spherical coordinates $ \boldsymbol{\phi=}(\phi_{1},\phi_{2})^{T}\in S^{1}\times S^{1} $, we can write parametrize the curve on a three-dimensional sphere by:



$$ \mathbf{e}(\boldsymbol{\phi})=\left(\begin{array}{c}
\cos\phi_{1}(s)\sin\phi_{2}(s)\\
\sin\phi_{1}(s)\sin\phi_{2}(s)\\
\cos\phi_{2}(s)
\end{array}\right),\quad\mathbf{A}:=D_{\boldsymbol{\phi}}\mathbf{e}=\left(\begin{array}{cc}
-\sin\phi_{1}\sin\phi_{2} & \cos\phi_{1}\cos\phi_{2}\\
\cos\phi_{1}\sin\phi_{2} & \sin\phi_{1}\cos\phi_{2}\\
0 & -\sin\phi_{2}
\end{array}\right) $$.

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add Algorithm folder to current working path
sys.path.append(parent_directory+"/subfunctions")

In [ ]:
from math import cos, sin, sqrt, pi
import numpy as np

# Import package to compute eigenvalue
from ipynb.fs.defs.eigen import eigen

def _tensorline_equation(t, x_phi, interpS11, interpS12, interpS13, interpS22, interpS23, interpS33, aux_grid, idx_eigenvector = 0):
    
    phi = x_phi[3]
    theta = x_phi[4]

    x = x_phi[0]%(2*pi)
    y = x_phi[1]%(2*pi)
    z = x_phi[2]%(2*pi)
    
    S11 = interpS11([y, x, z])[0]
    S12 = interpS12([y, x, z])[0]
    S22 = interpS22([y, x, z])[0]
    S13 = interpS13([y, x, z])[0]
    S23 = interpS23([y, x, z])[0]
    S33 = interpS33([y, x, z])[0]
    
    S21 = S12.copy()
    S31 = S13.copy()
    S32 = S23.copy()
    
    S = np.array([[S11, S12, S13], [S21, S22, S23], [S31, S32, S33]])

    lam, eigenv = eigen(S)
    
    # Define auxilary meshgrid
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    rho_z = aux_grid[2]
    
    xL = (x-rho_x)%(2*pi)
    xR = (x+rho_x)%(2*pi)
            
    yD = (y-rho_y)%(2*pi)
    yU = (y+rho_y)%(2*pi)
            
    zB = (z-rho_z)%(2*pi)
    zF = (z+rho_z)%(2*pi)
    
    # compute derivatives using auxiliary grid and finite-differencing
    S11x = (interpS11([y, xR, z])[0]-interpS11([y, xL, z])[0])/(2*rho_x)
    S11y = (interpS11([yU, x,z])[0]-interpS11([yD, x, z])[0])/(2*rho_y)
    S11z = (interpS11([y, x, zF])[0]-interpS11([y, x, zB])[0])/(2*rho_z)
        
    S12x = (interpS12([y, xR, z])[0]-interpS12([y, xL, z])[0])/(2*rho_x)
    S12y = (interpS12([yU, x, z])[0]-interpS12([yD, x, z])[0])/(2*rho_y)
    S12z = (interpS12([y, x, zF])[0]-interpS12([y, x, zB])[0])/(2*rho_z)
            
    S22x = (interpS22([y, xR, z])[0]-interpS22([y, xL, z])[0])/(2*rho_x)
    S22y = (interpS22([yU, x, z])[0]-interpS22([yD, x, z])[0])/(2*rho_y)
    S22z = (interpS22([y, x, zF])[0]-interpS22([y, x, zB])[0])/(2*rho_z)
            
    S33x = (interpS33([y, xR, z])[0]-interpS33([y, xL, z])[0])/(2*rho_x)
    S33y = (interpS33([yU, x, z])[0]-interpS33([yD, x, z])[0])/(2*rho_y)
    S33z = (interpS33([y, x, zF])[0]-interpS33([y, x, zB])[0])/(2*rho_z)
            
    S23x = (interpS23([y, xR, z])[0]-interpS23([y, xL, z])[0])/(2*rho_x)
    S23y = (interpS23([yU, x, z])[0]-interpS23([yD, x, z])[0])/(2*rho_y)
    S23z = (interpS23([y, x, zF])[0]-interpS23([y, x, zB])[0])/(2*rho_z)
            
    S13x = (interpS13([y, xR, z])[0]-interpS13([y, xL, z])[0])/(2*rho_x)
    S13y = (interpS13([yU, x, z])[0]-interpS13([yD, x, z])[0])/(2*rho_y)
    S13z = (interpS13([y, x, zF])[0]-interpS13([y, x, zB])[0])/(2*rho_z)
            
    e = np.array([[cos(phi)*sin(theta)], [sin(phi)*sin(theta)], [cos(theta)]])
    A = np.array([[-sin(phi)*sin(theta), cos(phi)*cos(theta)], [cos(phi)*sin(theta), sin(phi)*cos(theta)], [0,-sin(theta)]])
                            
    grad_Sx_e = [S11x*cos(phi)*sin(theta)+S11y*sin(phi)*sin(theta)+S11z*cos(theta), S12x*cos(phi)*sin(theta)+S12y*sin(phi)*sin(theta)+S12z*cos(theta), S13x*cos(phi)*sin(theta)+S13y*sin(phi)*sin(theta)+S13z*cos(theta)]
    grad_Sy_e = [S12x*cos(phi)*sin(theta)+S12y*sin(phi)*sin(theta)+S12z*cos(theta), S22x*cos(phi)*sin(theta)+S22y*sin(phi)*sin(theta)+S22z*cos(theta), S23x*cos(phi)*sin(theta)+S23y*sin(phi)*sin(theta)+S23z*cos(theta)]
    grad_Sz_e = [S13x*cos(phi)*sin(theta)+S13y*sin(phi)*sin(theta)+S13z*cos(theta), S23x*cos(phi)*sin(theta)+S23y*sin(phi)*sin(theta)+S23z*cos(theta), S33x*cos(phi)*sin(theta)+S33y*sin(phi)*sin(theta)+S33z*cos(theta)]
    
    grad_S_e = np.array([grad_Sx_e, grad_Sy_e, grad_Sz_e])
    
    f = -A.transpose()@(grad_S_e@e)
    M = A.transpose()@(S-lam[idx_eigenvector]*np.eye(3,3))@A
        
    M_inverse = np.linalg.inv(M)
    phi_theta_dot = (M_inverse@f).ravel()
        
    phi_dot = phi_theta_dot[0]
    theta_dot = phi_theta_dot[1]
        
    x_dot = cos(phi)*sin(theta)
    y_dot = sin(phi)*sin(theta)
    z_dot = cos(theta)
        
    norm = sqrt(1+theta_dot**2+phi_dot**2)
        
    return [x_dot/norm, y_dot/norm, z_dot/norm, phi_dot/norm, theta_dot/norm]    